In [156]:
import pandas as pd
import numpy as np
import importlib
import amazonprocessing
importlib.reload(amazonprocessing)

<module 'amazonprocessing' from 'e:\\Labmentrix\\Projects\\Project 3\\amazonprocessing.py'>

**Cleaning Data using amazonpreprocessing Module**

In [157]:
amazon=pd.read_csv("amazon_delivery.csv")

In [158]:
amazon=amazonprocessing.amazon_preprocess(amazon)

In [159]:
amazon

,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Weather,Traffic,Vehicle,Area,Delivery_Time,Category,Distance_km,Delay,Order_day,Pickup_day_part
Order_ID,,,,,,,,,,,,,,,,
ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,Sunny,High,motorcycle,Urban,120,Clothing,3.025149,15,5,Morning
akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,Stormy,Jam,scooter,Metropolitian,165,Electronics,20.183530,5,4,Evening
njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,Sandstorms,Low,motorcycle,Urban,130,Sports,1.552758,15,5,Morning
rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,Sunny,Medium,motorcycle,Metropolitian,105,Cosmetics,7.790401,10,1,Evening
zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,Cloudy,High,scooter,Metropolitian,150,Toys,6.210138,15,5,Afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
jlxf819993117,30,4.8,26.902328,75.794257,26.912328,75.804257,Windy,High,motorcycle,Metropolitian,160,Home,1.489846,10,3,Morning
aevx342135787,21,4.6,0.000000,0.000000,0.070000,0.070000,Windy,Jam,motorcycle,Metropolitian,180,Jewelry,11.007735,15,2,Evening
xnek760674819,30,4.9,13.022394,80.242439,13.052394,80.272439,Cloudy,Low,scooter,Metropolitian,80,Home,4.657195,15,4,Night


In [161]:
amazon.head(2)

,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Weather,Traffic,Vehicle,Area,Delivery_Time,Category,Distance_km,Delay,Order_day,Pickup_day_part
Order_ID,,,,,,,,,,,,,,,,
ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,Sunny,High,motorcycle,Urban,120,Clothing,3.025149,15,5,Morning
akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,Stormy,Jam,scooter,Metropolitian,165,Electronics,20.183530,5,4,Evening


In [162]:
amazon.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43739 entries, ialx566343618 to nsyz997960170
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Agent_Age        43739 non-null  int64   
 1   Agent_Rating     43739 non-null  float64 
 2   Store_Latitude   43739 non-null  float64 
 3   Store_Longitude  43739 non-null  float64 
 4   Drop_Latitude    43739 non-null  float64 
 5   Drop_Longitude   43739 non-null  float64 
 6   Weather          43739 non-null  object  
 7   Traffic          43739 non-null  object  
 8   Vehicle          43739 non-null  object  
 9   Area             43739 non-null  object  
 10  Delivery_Time    43739 non-null  int64   
 11  Category         43739 non-null  object  
 12  Distance_km      43739 non-null  float64 
 13  Delay            43739 non-null  int64   
 14  Order_day        43739 non-null  int32   
 15  Pickup_day_part  43739 non-null  category
dtypes: category(1), float64(6

In [163]:
amazon.columns

Index(['Agent_Age', 'Agent_Rating', 'Store_Latitude', 'Store_Longitude',
       'Drop_Latitude', 'Drop_Longitude', 'Weather', 'Traffic', 'Vehicle',
       'Area', 'Delivery_Time', 'Category', 'Distance_km', 'Delay',
       'Order_day', 'Pickup_day_part'],
      dtype='object')

In [164]:
amazon.shape

(43739, 16)

# Model Building

## Preprocessing Data and scaling

In [165]:
amazon.select_dtypes(["object","category"])

,Weather,Traffic,Vehicle,Area,Category,Pickup_day_part
Order_ID,,,,,,
ialx566343618,Sunny,High,motorcycle,Urban,Clothing,Morning
akqg208421122,Stormy,Jam,scooter,Metropolitian,Electronics,Evening
njpu434582536,Sandstorms,Low,motorcycle,Urban,Sports,Morning
rjto796129700,Sunny,Medium,motorcycle,Metropolitian,Cosmetics,Evening
zguw716275638,Cloudy,High,scooter,Metropolitian,Toys,Afternoon
...,...,...,...,...,...,...
jlxf819993117,Windy,High,motorcycle,Metropolitian,Home,Morning
aevx342135787,Windy,Jam,motorcycle,Metropolitian,Jewelry,Evening
xnek760674819,Cloudy,Low,scooter,Metropolitian,Home,Night


### Encoding Categorical data using Labelencoder

In [173]:
label_columns=amazon.select_dtypes(["object","category"]).columns

In [191]:
x_numeric_columns=amazon.select_dtypes(["float","int"]).columns.drop("Delivery_Time")
y_numeric_column="Delivery_Time"

In [193]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
categorical_encoder=OrdinalEncoder()
scaled=pd.DataFrame()
scaled[label_columns]=pd.DataFrame(categorical_encoder.fit_transform(amazon[label_columns]), columns=label_columns)

In [194]:
x_scaler=StandardScaler()
y_scaler=StandardScaler()

In [195]:
import mlflow
import mlflow.sklearn
